In [1]:
# Installing libraries: 
# pip install geopy 

In [2]:
# 1
# removing duplciate lane entries
# removing lane info "_lane" from dataset
import pandas as pd

def clean_dataset(df):
    # Group the data by the two columns and keep only the first occurrence of each group
    reduced_df = df.drop_duplicates(subset=[5, 6], keep='first')
    # Convert column 1 values to strings and remove anything after and including the last underscore
    #reduced_df[1] = reduced_df[1].astype(str).str.rsplit('_', n=1).str[0]
    reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]

    reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
    reduced_df.iloc[:, 6] = reduced_df.iloc[:, 6].astype(str).apply(lambda x: float(x[:4] + '.' + x[4:]))

    return reduced_df


In [3]:
# 2
# group data based on sensor location description 
import pandas as pd

def get_sensor_group(data, sensor_loc, ignore=None):
    # Filter the DataFrame based on column 3 and ignore string
    mask = data[3].str.startswith(sensor_loc) & (~data[3].str.contains(ignore) if ignore is not None else True)
    matching_group = data[mask]
    return matching_group


In [4]:
# Connecting the sensor groups based on travelling-salesman-problem algorithm:
# Its working as intended

import pandas as pd
import folium
from geopy.distance import distance

def calculate_distance(sensor_data):
    # Calculate distances between each pair of sensors
    num_sensors = len(sensor_data)
    distances = [[0.0] * num_sensors for _ in range(num_sensors)]
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            coord1 = (sensor_data.iloc[i][5], sensor_data.iloc[i][6])
            coord2 = (sensor_data.iloc[j][5], sensor_data.iloc[j][6])
            dist = distance(coord1, coord2).meters
            distances[i][j] = dist
            distances[j][i] = dist
    return distances

def tsp(sensor_data):
    
    distances = calculate_distance(sensor_data)
    map = folium.Map(location=[sensor_data[5].mean(), sensor_data[6].mean()], zoom_start=10)

    # Connect sensors with the shortest distance
    num_sensors = len(sensor_data)
    connected = set()
    min_distance = float('inf')
    min_i, min_j = -1, -1

    # Find the pair of sensors with the shortest distance
    for i in range(num_sensors):
        for j in range(i+1, num_sensors):
            if distances[i][j] < min_distance:
                min_distance = distances[i][j]
                min_i, min_j = i, j

    # Connect the pair of sensors with a polyline
    if min_i != -1 and min_j != -1:
        sensor1 = sensor_data.iloc[min_i]
        sensor2 = sensor_data.iloc[min_j]
        folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                        color='red').add_to(map)
        connected.add(min_i)
        connected.add(min_j)

    # Connect the remaining sensors
    while len(connected) < num_sensors:
        min_distance = float('inf')
        min_i = -1
        for i in range(num_sensors):
            if i not in connected:
                for j in connected:
                    if distances[i][j] < min_distance:
                        min_distance = distances[i][j]
                        min_i = i
                        min_j = j
        if min_i != -1:
            sensor1 = sensor_data.iloc[min_i]
            sensor2 = sensor_data.iloc[min_j]
            folium.PolyLine([(sensor1[5], sensor1[6]), (sensor2[5], sensor2[6])],
                            color='red').add_to(map)
            connected.add(min_i)

    
    for _, sensor in sensor_data.iterrows():
        tooltip_text = f"Sensor_ID: {sensor[1]}, Des: {sensor[3]}"
        folium.CircleMarker(
            location=[sensor[5], sensor[6]],
            radius=5,
            color='blue',
            fill=True,
            fill_color='blue',
            tooltip=tooltip_text  # Add tooltip
            ).add_to(map)
    return map



In [5]:
# Run the full process

# def main():

# Read from the original data and claen it:
data = clean_dataset(pd.read_csv('../detector/detectors2018.csv', header=None))

# Select sensor group:

#sensor_loc = "CC-215 WB"
#sensor_loc="CC-215 EB"
sensor_loc="I-15 NB"
#sensor_loc= "I-515 SB"
#sensor_loc="US-95 SB"

# Ignore "-" from the sensor group
ignore = 'Ramp'

# Get list of sensors using the sensor group
sensor_group = get_sensor_group(data, sensor_loc, ignore)

# Plot the map of the sensor group 

#sensor_map = plot_dfs(sensor_group) 
#sensor_map = plot_direc(sensor_group)
#sensor_map = plot_tsp(sensor_group)
#sensor_map = plot_min_dis(sensor_group)

sensor_map = tsp(sensor_group)
sensor_map

C:\Users\MSI\AppData\Local\Temp\ipykernel_20120\1195696214.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.loc[:, 1] = reduced_df.loc[:, 1].astype(str).str.rsplit('_', n=1).str[0]
C:\Users\MSI\AppData\Local\Temp\ipykernel_20120\1195696214.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reduced_df.iloc[:, 5] = reduced_df.iloc[:, 5].astype(str).apply(lambda x: float(x[:2] + '.' + x[2:]))
C:\Users\MSI\AppData\Local\Temp\ipykernel_20120\1195696214.py:14: SettingWithCopyWarning: 
A value is 

In [ ]:
# Dataset Preparation

# Check if sensor-data exits:

# Read from the original data and claen it:
data = clean_dataset(pd.read_csv('../detector/detectors2018.csv', header=None))
# Get the list of sensors using the sensor group name:
sensor_loc="I-15 NB"
ignore = '@'
sensor_group = get_sensor_group(data, sensor_loc, ignore)
search=sensor_group[1]
length=(len(search))

print(length)
for sensor_name in search:
    # Check if the CSV file exists in the folder with the sensor_name
    pass
    


In [10]:
sensor_group

,0,1,2,3,4,5,6,7,8,9,10
8,9,101_1_35,3,I-15 NB Sahara On-Ramp between Sahara Avenue a...,video Image,36.144341,-115.169617,580,1353,1842,4633
23,24,110_1_41,10,I-15 NB between I-15 NB Sahara On-Ramp and I-1...,microwave radar,36.150986,-115.163765,580,1353,1842,4633
33,34,112_2_44,12,I-15 NB between I-15 NB Charleston EB Off-Ramp...,microwave radar,36.156330,-115.159843,580,1353,1842,4633
38,39,113_2_45,13,I-15 NB between I-15 NB Charleston WB Off-Ramp...,microwave radar,36.159657,-115.159103,580,1353,1842,4633
41,42,114_2_44,14,I-15 NB Charleston Off-Ramp between I-15 NB an...,microwave radar,36.156761,-115.159393,580,1353,1842,4633
...,...,...,...,...,...,...,...,...,...,...,...
2319,2320,90_2_30,823,I-15 NB Spring Mtn WB On-Ramp between Spring M...,video Image,36.127480,-115.178146,580,1353,1842,4633
2334,2335,97_1_33,829,I-15 NB between I-15 NB Desert Inn On-Ramp and...,inductive loop,36.130348,-115.179474,580,1353,1842,4633
2340,2341,97_2_33,830,I-15 NB between I-15 NB Desert Inn On-Ramp and...,microwave radar,36.133587,-115.178215,580,1353,1842,4633
2346,2347,97_3_38,831,I-15 NB between I-15 NB Desert Inn On-Ramp and...,microwave radar,36.138489,-115.174706,580,1353,1842,4633
